# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd
doc = []
labels = ['index', 'text']
with open('dates.txt') as file:
    for line in file:
        doc.append(line.strip())

df = pd.Series(doc)
df = df.to_frame()
df.head(5)

,0
0,03/25/93 Total time of visit (in minutes):
1,6/18/85 Primary Care Doctor:
2,sshe plans to move as of 7/8/71 In-Home Servic...
3,7 on 9/27/75 Audit C Score Current:
4,2/6/96 sleep studyPain Treatment Pain Level (N...


In [2]:
index = list(range(len(df)))
df['index'] = index
df = df.rename(columns={df.columns[0]: 'text'.strip()})
df.head(5)

,text,index
0,03/25/93 Total time of visit (in minutes):,0
1,6/18/85 Primary Care Doctor:,1
2,sshe plans to move as of 7/8/71 In-Home Servic...,2
3,7 on 9/27/75 Audit C Score Current:,3
4,2/6/96 sleep studyPain Treatment Pain Level (N...,4


In [3]:
df['len'] = df['text'].str.len()
df.head(5)

,text,index,len
0,03/25/93 Total time of visit (in minutes):,0,42
1,6/18/85 Primary Care Doctor:,1,28
2,sshe plans to move as of 7/8/71 In-Home Servic...,2,54
3,7 on 9/27/75 Audit C Score Current:,3,35
4,2/6/96 sleep studyPain Treatment Pain Level (N...,4,62


In [4]:
df['text'].str.findall(r'\d')

0      [0, 3, 2, 5, 9, 3]
1         [6, 1, 8, 8, 5]
2            [7, 8, 7, 1]
3      [7, 9, 2, 7, 7, 5]
4         [2, 6, 9, 6, 7]
              ...        
495          [1, 9, 7, 9]
496          [2, 0, 0, 6]
497          [2, 0, 0, 8]
498          [2, 0, 0, 5]
499          [1, 9, 8, 0]
Name: text, Length: 500, dtype: object

In [5]:
df.tail(10)

,text,index,len
490,Lab: B12 969 2007,490,17
491,)and 8mo in 2009,491,16
492,.Moved to USA in 1986. Suffered from malnutrit...,492,93
493,r1978,493,5
494,". Went to Emerson, in Newfane Alaska. Started ...",494,171
495,1979 Family Psych History: Family History of S...,495,68
496,therapist and friend died in ~2006 Parental/Ca...,496,66
497,2008 partial thyroidectomy,497,26
498,sPt describes a history of sexual abuse as a c...,498,285
499,". In 1980, patient was living in Naples and de...",499,363


In [15]:
#df['extract'] = df['text'].str.extractall(r'(?:\d{1,2})[?:/](?:\d{1,2})[?:/]\d{2,4}')
#Excellent regex visualization source
#https://jex.im/regulex/#!flags=&re=(r'((%5Cd%7B1%2C2%7D%3F)%5B%3F%2F%5D(%5Cd%7B1%2C2%7D%3F)%5B%3F%2F%5D(%5Cd%7B2%2C4%7D))')

df['text'].str.extractall((r'((\d{1,2}?)[/](\d{1,2}?)[/](\d{2,4}))'))


,,0,1,2,3
,match,,,,
0,0,03/25/93,03,25,93
1,0,6/18/85,6,18,85
2,0,7/8/71,7,8,71
3,0,9/27/75,9,27,75
4,0,2/6/96,2,6,96
...,...,...,...,...,...
120,0,12/22/98,12,22,98
121,0,10/02/96,10,02,96
122,0,11/05/90,11,05,90


In [16]:
df[100:110]

,text,index,len
100,8/16/92 SOS-10 Total Score:,100,27
101,10/05/97 CPT Code: 90791: No medical services,101,45
102,07/18/2002 CPT Code: 90792: With medical services,102,49
103,9/22/82 Total time of visit (in minutes):,103,41
104,2/24/74 SOS-10 Total Score:,104,27
105,(2/03/78) TSH-0.90 Activities of Daily Living ...,105,72
106,2/11/2006 CPT Code: 90791: No medical services,106,46
107,"Pt is a 21 year old, single, heterosexual iden...",107,1071
108,PET Scan (DPSH 5/04/74): 1) Marked hypometabol...,108,473
109,7/20/2011 [report_end],109,22


In [ ]:
def date_sorter():
    
    order = None
    # YOUR CODE HERE
    raise NotImplementedError()
    return order # Your answer here